In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Part 1: Cost Report data

Source: The Skilled Nursing Facility Cost Report forms, a highly technical pdf report that is sent to CMS and turned into a machine-readable database on a yearly basis. [Link](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Cost-Reports/Cost-Reports-by-Fiscal-Year.html)

The data is highly vertical, with the basic layout being that each row specifies the page and section of the pdf and the value for each nursing home report. The advantage is that this is more accurate for providing information such as staffing levels; it's used in the tool to give the staffing hours for each nursing home.

In [2]:
#STEP 1. Process metadata on the reports
#  reports folder connects NPI to report number
# a. read in csv
rp = pd.read_csv('SNF10FY2017/snf10_2017_RPT.CSV', header=None, names=['RPT_REC_NUM', 
'PRVDR_CTRL_TYPE_CD', 
'NPI', 
'PRVDR_NUM', 
'RPT_STUS_CD', 
'FY_BGN_DT', 
'FY_END_DT', 
'PROC_DT', 
'INITL_RPT_SW', 
'LAST_RPT_SW', 
'TRNSMTL_NUM', 
'FI_NUM', 
'ADR_VNDR_CD', 
'FI_CREAT_DT', 
'UTIL_CD', 
'NPR_DT', 
'SPEC_IND', 
'FI_RCPT_DT'], index_col=False, dtype={'RPT_REC_NUM':str, 'NPI':str})

In [3]:
rp.head()

,RPT_REC_NUM,PRVDR_CTRL_TYPE_CD,NPI,PRVDR_NUM,RPT_STUS_CD,FY_BGN_DT,FY_END_DT,PROC_DT,INITL_RPT_SW,LAST_RPT_SW,TRNSMTL_NUM,FI_NUM,ADR_VNDR_CD,FI_CREAT_DT,UTIL_CD,NPR_DT,SPEC_IND,FI_RCPT_DT
0,1156385,5.0,225718,NaN,2,10/01/2016,10/31/2016,04/26/2017,N,N,NaN,5901,4,04/26/2017,F,04/25/2017,NaN,03/01/2017
1,1157220,4.0,345507,NaN,2,10/01/2016,12/31/2016,05/15/2017,N,N,NaN,11501,4,05/09/2017,F,05/09/2017,NaN,05/03/2017
2,1157221,4.0,345222,NaN,2,10/01/2016,12/31/2016,05/15/2017,N,N,NaN,11501,4,05/09/2017,F,05/09/2017,NaN,05/03/2017
3,1157223,4.0,345567,NaN,2,10/01/2016,12/31/2016,05/15/2017,N,N,NaN,11501,4,05/10/2017,F,05/09/2017,NaN,05/03/2017
4,1158700,4.0,135038,NaN,1,10/01/2016,12/31/2016,05/31/2017,N,N,NaN,10001,4,05/23/2017,F,NaN,NaN,05/04/2017


In [4]:
# c. list of providers allows me to filter data to NJ
pro = pd.read_csv('July_2019_project/Provider_Info.csv', dtype={'provnum':str}).rename(columns={'Federal Provider Number':'provnum',
                                                                                          'Provider State':'STATE'})

pro = pro[['provnum', 'STATE']].drop_duplicates()

#merge with report file to get record nos.
st_rp = rp.merge(pro, how='left', left_on='NPI', right_on='provnum')

#limit to full-year data
# st_rp = st_rp[(st_rp['FY_BGN_DT'] == '01/01/2015') & (st_rp['FY_END_DT'] == '12/31/2015')]
st_npi = st_rp[['RPT_REC_NUM', 'NPI', 'STATE']]

In [5]:
st_npi.isnull().sum()

RPT_REC_NUM      0
NPI              0
STATE          216
dtype: int64

In [6]:
#reading in the text data
al = pd.read_csv('SNF10FY2017/snf10_2017_ALPHA.CSV', header=None, names = ['RPT_REC_NUM', 'WKSHT_CD', 'LINE_NUM', 'CLMN_NUM', 'ITM_ALPHNMRC_ITM_TXT'], dtype={'RPT_REC_NUM':str, 'LINE_NUM':str, 'CLMN_NUM':str})

In [7]:
al.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_ALPHNMRC_ITM_TXT
0,1156385,A000000,00100,00000,0100CAP REL COSTS - BLDGS & FIXTURES
1,1156385,A000000,00200,00000,0200CAP REL COSTS - MOVABLE EQUIPMEN
2,1156385,A000000,00300,00000,0300EMPLOYEE BENEFITS
3,1156385,A000000,00400,00000,0400ADMINISTRATIVE & GENERAL
4,1156385,A000000,00500,00000,0500PLANT OPERATION MAINT. & REPAIR


In [8]:
# filter_df(al_npi_fil, 'alpha')

In [9]:
#b. adding the npi of the report number for just the homes we're looking for  - will become more clear later on
al_npi_fil = al.merge(st_npi, how='left', on='RPT_REC_NUM')

In [10]:
#same thing with numeric data
num = pd.read_csv('SNF10FY2017/snf10_2017_NMRC.CSV', header=None, names = ['RPT_REC_NUM', 'WKSHT_CD', 'LINE_NUM', 'CLMN_NUM', 'ITM_VAL_NUM'], dtype={'RPT_REC_NUM':str, 'LINE_NUM':str, 'CLMN_NUM':str})

In [11]:
num.head()

,RPT_REC_NUM,WKSHT_CD,LINE_NUM,CLMN_NUM,ITM_VAL_NUM
0,1156385,A000000,00100,00200,84299.0
1,1156385,A000000,00100,00300,84299.0
2,1156385,A000000,00100,00500,84299.0
3,1156385,A000000,00100,00600,736.0
4,1156385,A000000,00100,00700,85035.0


In [12]:
# filter_df(num_npi, 'num')

In [13]:
#b. 
num_npi_fil = num.merge(st_npi, how='left', on='RPT_REC_NUM')

In [14]:
#now we filter the dataset to just the NPIs we want.
# al_spec = al_npi_fil[al_npi_fil['NPI'].apply(lambda x: x in npis)]
# num_spec = num_npi_fil[num_npi_fil['NPI'].apply(lambda x: x in npis)]

In [15]:
#c. get state data + create NJ dataset
# first have to add state of each
al_nj_npi = al.merge(st_npi, how='left', on='RPT_REC_NUM')
num_nj_npi = num.merge(st_npi, how='left', on='RPT_REC_NUM')

In [16]:
#Just NJ values
al_nj = al_npi_fil[al_npi_fil['STATE'] == 'NJ']
num_nj = num_npi_fil[num_npi_fil['STATE'] == 'NJ']

In [17]:
al_nj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88872 entries, 25183 to 3404151
Data columns (total 7 columns):
RPT_REC_NUM             88872 non-null object
WKSHT_CD                88872 non-null object
LINE_NUM                88872 non-null object
CLMN_NUM                88872 non-null object
ITM_ALPHNMRC_ITM_TXT    88814 non-null object
NPI                     88872 non-null object
STATE                   88872 non-null object
dtypes: object(7)
memory usage: 5.4+ MB


In [18]:
#STEP 3: Create dataframe with staffing hours for all homes, NJ homes, and our 10 homes

#a. number of inpatient days for all residents
num_inp = num[num['WKSHT_CD'] == 'S300001']
num_inp = num_inp[num_inp['LINE_NUM'] == '00100']
num_inp = num_inp[num_inp['CLMN_NUM'] == '00700']
num_inp['ITM_VAL_NUM'].median()

26605.5

#### In this section, I filter down the dataframe to look at the total number of hours worked for each type of nurse, then divide that by the number of patients to get the number of hours per patient per day for different types of nursing in the home.

In [19]:
#b. hours for each category of nurses
# RN hours for all NHs
num_rn = num[num['WKSHT_CD'] == 'S300005']
num_rn = num_rn[(num_rn['LINE_NUM'] == '01400') | (num_rn['LINE_NUM'] == '00100') ]
num_rn = num_rn[num_rn['CLMN_NUM'] == '00400']
num_rn_piv = num_rn.pivot_table(index='RPT_REC_NUM', values='ITM_VAL_NUM', aggfunc=sum)
num_rn2 = pd.DataFrame(num_rn_piv)
num_rn2 = num_rn2.reset_index()

#lpns for all NHs
num_lpn = num[num['WKSHT_CD'] == 'S300005']
num_lpn = num_lpn[(num_lpn['LINE_NUM'] == '01500') | (num_lpn['LINE_NUM'] == '00200')]
num_lpn = num_lpn[num_lpn['CLMN_NUM'] == '00400']
num_lpn_piv = num_lpn.pivot_table(index='RPT_REC_NUM', values='ITM_VAL_NUM', aggfunc=sum)
num_lpn2 = pd.DataFrame(num_lpn_piv)
num_lpn2 = num_lpn2.reset_index()

#assistant hours for all NHs
num_asst = num[num['WKSHT_CD'] == 'S300005']
num_asst = num_asst[(num_asst['LINE_NUM'] == '01600') | (num_asst['LINE_NUM'] == '00300')]
num_asst = num_asst[num_asst['CLMN_NUM'] == '00400']
num_asst_piv = num_asst.pivot_table(index='RPT_REC_NUM', values='ITM_VAL_NUM', aggfunc=sum)
num_asst2 = pd.DataFrame(num_asst_piv)
num_asst2 = num_asst2.reset_index()

In [20]:
#c. calculate staffing hours per inpatient day by dividing two categories
# rn hours per patient day: all
num_rn_day = num_inp.merge(num_rn2, on='RPT_REC_NUM', suffixes=('_days','_rn'))
num_rn_day['rn_hr_per_day'] = num_rn_day['ITM_VAL_NUM_rn'] / num_rn_day['ITM_VAL_NUM_days'] 

num_lpn_day = num_inp.merge(num_lpn2, on='RPT_REC_NUM', suffixes=('_days', '_lpn'))
num_lpn_day['lpn_hr_day'] = num_lpn_day['ITM_VAL_NUM_lpn'] / num_lpn_day['ITM_VAL_NUM_days']

#asst hours per patient day: all
num_asst_day = num_inp.merge(num_asst2, on='RPT_REC_NUM', suffixes=('_days','_asst'))
num_asst_day['asst_hr_per_day'] = num_asst_day['ITM_VAL_NUM_asst'] / num_asst_day['ITM_VAL_NUM_days'] 

In [21]:
#d. merge RN, LPN and Aide datasets into one nice dataframe
all_num_hr_day = num_rn_day.merge(num_lpn_day,on='RPT_REC_NUM').merge(num_asst_day,on='RPT_REC_NUM')
all_num_hr_day = all_num_hr_day[[u'RPT_REC_NUM', u'ITM_VAL_NUM_days_x', u'ITM_VAL_NUM_rn', u'rn_hr_per_day',
       u'ITM_VAL_NUM_lpn', u'lpn_hr_day', u'ITM_VAL_NUM_asst', u'asst_hr_per_day']]
all_num_hr_day = all_num_hr_day.rename(columns={'ITM_VAL_NUM_days_x':'inpatient_days', 'ITM_VAL_NUM_rn':'rn_hours', 
                                               'ITM_VAL_NUM_lpn':'lpn_hours', 'ITM_VAL_NUM_asst':'asst_hours'})

In [22]:
#a. number of inpatient days for NJ nhs
num_inp_nj = num_nj[num_nj['WKSHT_CD'] == 'S300001']
num_inp_nj = num_inp_nj[num_inp_nj['LINE_NUM'] == '00100']
num_inp_nj = num_inp_nj[num_inp_nj['CLMN_NUM'] == '00700']

#b. RN hours for NJ NHs
num_rn_nj = num_nj[num_nj['WKSHT_CD'] == 'S300005']
num_rn_nj = num_rn_nj[(num_rn_nj['LINE_NUM'] == '01400') | (num_rn_nj['LINE_NUM'] == '00100') ]
num_rn_nj = num_rn_nj[num_rn_nj['CLMN_NUM'] == '00400']
num_rn_nj_piv = num_rn_nj.pivot_table(index=('RPT_REC_NUM', 'NPI'), values='ITM_VAL_NUM', aggfunc=sum)
num_rn_nj2 = pd.DataFrame(num_rn_nj_piv)
num_rn_nj2 = num_rn_nj2.reset_index()

#b. lpns for NJ NHs
num_lpn_nj = num_nj[num_nj['WKSHT_CD'] == 'S300005']
num_lpn_nj = num_lpn_nj[(num_lpn_nj['LINE_NUM'] == '01500') | (num_lpn_nj['LINE_NUM'] == '00200')]
num_lpn_nj = num_lpn_nj[num_lpn_nj['CLMN_NUM'] == '00400']
num_lpn_nj_piv = num_lpn_nj.pivot_table(index='RPT_REC_NUM', values='ITM_VAL_NUM', aggfunc=sum)
num_lpn_nj2 = pd.DataFrame(num_lpn_nj_piv)
num_lpn_nj2 = num_lpn_nj2.reset_index()


#b. assistant hours for NJ NHs
num_asst_nj = num_nj[num_nj['WKSHT_CD'] == 'S300005']
num_asst_nj = num_asst_nj[(num_asst_nj['LINE_NUM'] == '01600') | (num_asst_nj['LINE_NUM'] == '00300')]
num_asst_nj = num_asst_nj[num_asst_nj['CLMN_NUM'] == '00400']
num_asst_nj['ITM_VAL_NUM'].median()
num_asst_nj_piv = num_asst_nj.pivot_table(index='RPT_REC_NUM', values='ITM_VAL_NUM', aggfunc=sum)
num_asst_nj2 = pd.DataFrame(num_asst_nj_piv)
num_asst_nj2 = num_asst_nj2.reset_index()

In [23]:
#c. creating the per-day calculations
num_rn_day_nj = num_inp_nj.merge(num_rn_nj2, on='RPT_REC_NUM', suffixes=('_days','_rn'))
num_rn_day_nj['rn_hr_per_day'] = num_rn_day_nj['ITM_VAL_NUM_rn'] / num_rn_day_nj['ITM_VAL_NUM_days'] 

num_lpn_day_nj = num_inp_nj.merge(num_lpn_nj2, on='RPT_REC_NUM', suffixes=('_days', '_lpn'))
num_lpn_day_nj['lpn_hr_day'] = num_lpn_day_nj['ITM_VAL_NUM_lpn'] / num_lpn_day_nj['ITM_VAL_NUM_days']

num_asst_day_nj = num_inp_nj.merge(num_asst_nj2, on='RPT_REC_NUM', suffixes=('_days','_asst'))
num_asst_day_nj['asst_hr_per_day'] = num_asst_day_nj['ITM_VAL_NUM_asst'] / num_asst_day_nj['ITM_VAL_NUM_days'] 



In [24]:
#d. putting it all together into one nice file
nj_num_hr_day = num_rn_day_nj.merge(num_lpn_day_nj,on='RPT_REC_NUM').merge(num_asst_day_nj,on='RPT_REC_NUM')
nj_num_hr_day = nj_num_hr_day[[u'RPT_REC_NUM', u'ITM_VAL_NUM_days_x', u'ITM_VAL_NUM_rn', u'rn_hr_per_day',
       u'ITM_VAL_NUM_lpn', u'lpn_hr_day', u'ITM_VAL_NUM_asst', u'asst_hr_per_day', 'NPI_rn']]
nj_num_hr_day = nj_num_hr_day.rename(columns={'ITM_VAL_NUM_days_x':'inpatient_days', 'ITM_VAL_NUM_rn':'rn_hours', 
                                               'ITM_VAL_NUM_lpn':'lpn_hours', 'ITM_VAL_NUM_asst':'asst_hours'})

In [25]:
nj_num_hr_day['total_hours_per_day'] = (nj_num_hr_day['rn_hours'] + nj_num_hr_day['lpn_hours'] + nj_num_hr_day['asst_hours']) / nj_num_hr_day['inpatient_days']

In [26]:
nj_num_hr_day.head()

,RPT_REC_NUM,inpatient_days,rn_hours,rn_hr_per_day,lpn_hours,lpn_hr_day,asst_hours,asst_hr_per_day,NPI_rn,total_hours_per_day
0,1174398,4287.0,2593.0,0.604852,4934.0,1.150921,10459.0,2.439701,315480,4.195475
1,1177778,10408.0,4294.0,0.412567,11445.0,1.099635,23663.0,2.273540,315321,3.785742
2,1181775,9607.0,13540.0,1.409389,2185.0,0.227438,24349.0,2.534506,315404,4.171333
3,1181786,23241.0,21451.0,0.922981,12997.0,0.559227,62364.0,2.683361,315331,4.165569
4,1183357,78561.0,18054.0,0.229809,40766.0,0.518909,173476.0,2.208169,315248,2.956887


In [27]:
nj_num_hr_day.describe()

,inpatient_days,rn_hours,rn_hr_per_day,lpn_hours,lpn_hr_day,asst_hours,asst_hr_per_day,total_hours_per_day
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,42829.385294,32257.684588,0.839244,38069.278941,0.927243,101479.300441,2.464372,4.230858
std,21858.710849,21169.763684,0.599833,21631.106166,0.405812,54000.708179,0.794018,1.400135
min,2061.000000,2071.000000,0.119898,668.000000,0.073992,10459.000000,0.681553,2.235762
25%,30023.750000,16501.000000,0.467506,23378.250000,0.713519,68620.440000,2.092077,3.576881
50%,39678.500000,27751.000000,0.733998,34786.390000,0.900799,90025.000000,2.311916,3.945270
75%,54629.000000,42537.250000,1.007164,48331.500000,1.077676,124343.250000,2.617289,4.413987
max,191948.000000,124861.000000,5.812654,154942.000000,4.271100,371715.000000,10.763886,18.114233


In [28]:
#mean hours per day for new jersey
print nj_num_hr_day['rn_hours'].sum() / nj_num_hr_day['inpatient_days'].sum()
print nj_num_hr_day['lpn_hours'].sum() / nj_num_hr_day['inpatient_days'].sum()
print nj_num_hr_day['asst_hours'].sum() / nj_num_hr_day['inpatient_days'].sum()

0.753167115678
0.888858868269
2.3693849385


In [29]:
#median hours per day for NJ
print nj_num_hr_day['rn_hr_per_day'].median()
print nj_num_hr_day['lpn_hr_day'].median()
print nj_num_hr_day['asst_hr_per_day'].median()
print nj_num_hr_day['total_hours_per_day'].median()

0.733997691212
0.900798868701
2.31191567856
3.94526958475


In [30]:
nj_num_hr_day['tot_hours'] = nj_num_hr_day['rn_hours'] + nj_num_hr_day['lpn_hours'] + nj_num_hr_day['asst_hours']

In [31]:
nj_num_hr_day.corr()

,inpatient_days,rn_hours,rn_hr_per_day,lpn_hours,lpn_hr_day,asst_hours,asst_hr_per_day,total_hours_per_day,tot_hours
inpatient_days,1.000000,0.556070,-0.282001,0.730581,-0.185877,0.884441,-0.235087,-0.308005,0.900233
rn_hours,0.556070,1.000000,0.401868,0.352212,-0.108225,0.563748,0.014005,0.148739,0.707793
rn_hr_per_day,-0.282001,0.401868,1.000000,-0.229244,0.200292,-0.168324,0.445428,0.739066,-0.066213
lpn_hours,0.730581,0.352212,-0.229244,1.000000,0.416359,0.667519,-0.072618,-0.018716,0.778326
lpn_hr_day,-0.185877,-0.108225,0.200292,0.416359,1.000000,-0.069692,0.440106,0.625230,0.035273
asst_hours,0.884441,0.563748,-0.168324,0.667519,-0.069692,1.000000,0.161203,-0.000893,0.960444
asst_hr_per_day,-0.235087,0.014005,0.445428,-0.072618,0.440106,0.161203,1.000000,0.885487,0.088795
total_hours_per_day,-0.308005,0.148739,0.739066,-0.018716,0.625230,-0.000893,0.885487,1.000000,0.032213
tot_hours,0.900233,0.707793,-0.066213,0.778326,0.035273,0.960444,0.088795,0.032213,1.000000


In [32]:
nj_num_hr_day.describe()

,inpatient_days,rn_hours,rn_hr_per_day,lpn_hours,lpn_hr_day,asst_hours,asst_hr_per_day,total_hours_per_day,tot_hours
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,42829.385294,32257.684588,0.839244,38069.278941,0.927243,101479.300441,2.464372,4.230858,171806.263971
std,21858.710849,21169.763684,0.599833,21631.106166,0.405812,54000.708179,0.794018,1.400135,83684.518944
min,2061.000000,2071.000000,0.119898,668.000000,0.073992,10459.000000,0.681553,2.235762,17986.000000
25%,30023.750000,16501.000000,0.467506,23378.250000,0.713519,68620.440000,2.092077,3.576881,116688.750000
50%,39678.500000,27751.000000,0.733998,34786.390000,0.900799,90025.000000,2.311916,3.945270,155961.935000
75%,54629.000000,42537.250000,1.007164,48331.500000,1.077676,124343.250000,2.617289,4.413987,211547.500000
max,191948.000000,124861.000000,5.812654,154942.000000,4.271100,371715.000000,10.763886,18.114233,617708.000000


In [33]:
#f. the actual rank of each nursing home in terms of each category
nj_num_hr_day['CMS_RN_rank'] = nj_num_hr_day['rn_hr_per_day'].rank(ascending=False)
nj_num_hr_day['CMS_LPN_rank'] = nj_num_hr_day['lpn_hr_day'].rank(ascending=False)
nj_num_hr_day['CMS_aide_rank'] = nj_num_hr_day['asst_hr_per_day'].rank(ascending=False)
nj_num_hr_day['CMS_tot_rank'] = nj_num_hr_day['total_hours_per_day'].rank(ascending=False)

#f. the percentile rank of each nursing home in terms of each category
nj_num_hr_day['CMS_RN_rankpct'] = nj_num_hr_day['rn_hr_per_day'].rank(pct=True, ascending=False)
nj_num_hr_day['CMS_LPN_rankpct'] = nj_num_hr_day['lpn_hr_day'].rank( pct=True, ascending=False)
nj_num_hr_day['CMS_aide_rankpct'] = nj_num_hr_day['asst_hr_per_day'].rank(pct=True, ascending=False)
nj_num_hr_day['CMS_tot_rankpct'] = nj_num_hr_day['total_hours_per_day'].rank(pct=True, ascending=False)

## Part 2: Adding CMS Nursing Home Compare data

The CMS Nursing Home Compare datasets give information about the quality of each nursing home and rates those homes on a 5-star scale. [Link](https://data.medicare.gov/data/nursing-home-compare)

These are updated on a rolling basis, but I used yearly averages when available.

In [34]:
#basic info about name, address, etc. of each home

provider_all = pd.read_csv('July_2019_project/Provider_Info.csv')

In [35]:
provider_nj = provider_all[provider_all['Provider State'] == 'NJ'] 

In [36]:
provider_nj.shape

(364, 85)

In [37]:
#b. merging with CMS staffing data
st_cms = provider_nj.merge(nj_num_hr_day, left_on='Federal Provider Number', right_on='NPI_rn', how='left')

In [38]:
st_cms.columns

Index([u'Federal Provider Number', u'Provider Name', u'Provider Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Provider Phone Number', u'Provider SSA County Code',
       u'Provider County Name', u'Ownership Type',
       ...
       u'total_hours_per_day', u'tot_hours', u'CMS_RN_rank', u'CMS_LPN_rank',
       u'CMS_aide_rank', u'CMS_tot_rank', u'CMS_RN_rankpct',
       u'CMS_LPN_rankpct', u'CMS_aide_rankpct', u'CMS_tot_rankpct'],
      dtype='object', length=104)

In [39]:
st_cms.head()

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,...,total_hours_per_day,tot_hours,CMS_RN_rank,CMS_LPN_rank,CMS_aide_rank,CMS_tot_rank,CMS_RN_rankpct,CMS_LPN_rankpct,CMS_aide_rankpct,CMS_tot_rankpct
0,315002,CARE ONE AT SOMERSET VALLEY,1621 ROUTE 22 WEST,BOUND BROOK,NJ,8805,7324692000,350,Somerset,For profit - Corporation,...,4.856506,96965.00,23.0,221.0,115.0,55.0,0.067647,0.650000,0.338235,0.161765
1,315008,LAUREL MANOR HEALTHCARE AND REHABILITATION CENTER,18 W LAUREL ROAD,STRATFORD,NJ,8084,8567842400,160,Camden,For profit - Partnership,...,3.377357,105664.00,297.0,93.0,293.0,285.0,0.873529,0.273529,0.861765,0.838235
2,315015,MADISON CENTER,625 STATE HIGHWAY 34,MATAWAN,NJ,7747,7325666400,290,Monmouth,For profit - Corporation,...,4.024121,207757.34,90.0,161.0,249.0,158.0,0.264706,0.473529,0.732353,0.464706
3,315057,MERRY HEART NURSING HOME,200 RT 10 WEST,SUCCASUNNA,NJ,7876,9735844000,300,Morris,For profit - Partnership,...,4.613132,167286.00,65.0,255.0,59.0,64.0,0.191176,0.750000,0.173529,0.188235
4,315087,CARE ONE AT KING JAMES,1040 ROUTE 36,ATLANTIC HIGHLANDS,NJ,7716,7322913400,290,Monmouth,For profit - Corporation,...,4.303234,134248.00,191.0,115.0,77.0,107.0,0.561765,0.338235,0.226471,0.314706


In [40]:
#getting geographic data for each for the map
st_cms['Latlong'] = st_cms.Location.str.split("(", 
                                                            expand=True)[1].fillna(0)

st_cms['lat'] = st_cms['Latlong'].str.split(',', expand=True)[0]

st_cms['long'] = st_cms['Latlong'].str.split(',', expand=True)[1].str[:-1]



In [41]:
st_cms.columns[0:10]

Index([u'Federal Provider Number', u'Provider Name', u'Provider Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Provider Phone Number', u'Provider SSA County Code',
       u'Provider County Name', u'Ownership Type'],
      dtype='object')

In [42]:
# CMS NHC "Quality" measures for different metrics, like "how many times were patients restrained?"

quality = pd.read_csv('July_2019_project/MDS_Quality_Measures_2019.csv')

In [43]:
quality.columns

Index([u'Federal Provider Number', u'Provider Name', u'Provider Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Measure Code', u'Measure Description', u'Resident type',
       u'Q1 Measure Score', u'Footnote for Q1 Measure Score',
       u'Q2 Measure Score', u'Footnote for Q2 Measure Score',
       u'Q3 Measure Score', u'Footnote for Q3 Measure Score',
       u'Q4 Measure Score', u'Footnote for Q4 Measure Score',
       u'Four Quarter Average Score', u'Footnote for Four Quarter Average',
       u'Used in Quality Measure Five Star Rating', u'Measure Period',
       u'Location', u'Processing Date'],
      dtype='object')

In [44]:
quality['Measure Description'].unique()

array(['Percentage of short-stay residents who made improvements in function',
       'Percentage of long-stay residents who lose too much weight',
       'Percentage of short-stay residents who newly received an antipsychotic medication',
       'Percentage of long-stay residents who have depressive symptoms',
       'Percentage of long-stay residents who received an antianxiety or hypnotic medication',
       'Percentage of short-stay residents assessed and appropriately given the pneumococcal vaccine',
       'Percentage of long-stay residents who self-report moderate to severe pain',
       'Percentage of long-stay residents experiencing one or more falls with major injury',
       'Percentage of long-stay residents whose ability to move independently worsened',
       'Percentage of high risk long-stay residents with pressure ulcers',
       'Percentage of short-stay residents who self-report moderate to severe pain',
       'Percentage of long-stay residents whose need for help w

In [45]:
#filtering the dataset and creating a new dataframe of it 
quality = quality.set_index('Federal Provider Number')

quality_fall = quality[quality['Measure Description'] == 'Percentage of long-stay residents experiencing one or more falls with major injury'][['Four Quarter Average Score']].rename(columns={
        'Four Quarter Average Score':'Percent with fall'})
quality_res = quality[quality['Measure Description'] == 'Percentage of long-stay residents who were physically restrained'][['Four Quarter Average Score']].rename(columns={
        'Four Quarter Average Score':'Percent restrained'})
quality_anx = quality[quality['Measure Description'] == 'Percentage of long-stay residents who received an antianxiety or hypnotic medication'][['Four Quarter Average Score']].rename(columns={
        'Four Quarter Average Score':'Percent_antianxiety'})
quality_psy = quality[quality['Measure Description'] == 'Percentage of long-stay residents who received an antipsychotic medication'][['Four Quarter Average Score']].rename(columns={
        'Four Quarter Average Score':'Percent_antipsychotic'})

quality_short = pd.concat([quality_fall, quality_res, quality_anx, quality_psy], axis=1)
quality_short['Percent with psych meds'] = quality_short['Percent_antianxiety'] + quality_short['Percent_antipsychotic']
quality_short.head()

/Users/epetenko/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Percent with fall,Percent restrained,Percent_antianxiety,Percent_antipsychotic,Percent with psych meds
015009,2.105260,0.00000,44.705881,15.083800,59.789681
015010,4.329004,0.00000,28.019322,35.294120,63.313442
015012,11.267605,0.00000,47.107439,0.000000,47.107439
015014,1.277954,0.00000,14.332245,7.636363,21.968608
015015,1.457727,0.58309,47.703180,32.831325,80.534505


In [46]:
#merge that with whole dataset
st_cms = st_cms.merge(quality_short, left_on='Federal Provider Number', 
                     right_index=True, how='left')

st_cms['Fall_Rank'] = st_cms['Percent with fall'].rank()
st_cms['Restrained_rank'] = st_cms['Percent restrained'].rank()
st_cms['Psych_meds_rank'] = st_cms['Percent with psych meds'].rank()

st_cms.head()

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,...,lat,long,Percent with fall,Percent restrained,Percent_antianxiety,Percent_antipsychotic,Percent with psych meds,Fall_Rank,Restrained_rank,Psych_meds_rank
0,315002,CARE ONE AT SOMERSET VALLEY,1621 ROUTE 22 WEST,BOUND BROOK,NJ,8805,7324692000,350,Somerset,For profit - Corporation,...,NaN,NaN,0.000000,0.0,NaN,0.000000,NaN,14.0,140.0,NaN
1,315008,LAUREL MANOR HEALTHCARE AND REHABILITATION CENTER,18 W LAUREL ROAD,STRATFORD,NJ,8084,8567842400,160,Camden,For profit - Partnership,...,NaN,NaN,0.343642,0.0,16.296299,11.284047,27.580346,31.0,140.0,190.0
2,315015,MADISON CENTER,625 STATE HIGHWAY 34,MATAWAN,NJ,7747,7325666400,290,Monmouth,For profit - Corporation,...,NaN,NaN,2.460849,0.0,24.065422,12.732096,36.797518,191.0,140.0,282.0
3,315057,MERRY HEART NURSING HOME,200 RT 10 WEST,SUCCASUNNA,NJ,7876,9735844000,300,Morris,For profit - Partnership,...,NaN,NaN,6.153846,0.0,16.872428,9.836067,26.708495,344.0,140.0,179.0
4,315087,CARE ONE AT KING JAMES,1040 ROUTE 36,ATLANTIC HIGHLANDS,NJ,7716,7322913400,290,Monmouth,For profit - Corporation,...,NaN,NaN,2.713178,0.0,25.352110,10.358567,35.710677,217.0,140.0,270.0


In [47]:
quality_short.describe()

,Percent with fall,Percent restrained,Percent_antianxiety,Percent_antipsychotic,Percent with psych meds
count,15032.000000,15031.000000,15003.000000,15006.000000,14978.000000
mean,3.380767,0.280020,20.417168,14.561742,34.957373
std,2.407205,1.755297,10.219210,9.900044,16.338376
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.646091,0.000000,13.138685,8.427782,24.150856
50%,3.021148,0.000000,19.245285,12.982098,32.758747
75%,4.651161,0.000000,26.153846,18.452380,42.772493
max,26.422765,98.104266,86.764705,100.000000,160.394394


In [50]:
st_cms.columns[:12]

Index([u'Federal Provider Number', u'Provider Name', u'Provider Address',
       u'Provider City', u'Provider State', u'Provider Zip Code',
       u'Provider Phone Number', u'Provider SSA County Code',
       u'Provider County Name', u'Ownership Type', u'Number of Certified Beds',
       u'Average Number of Residents Per Day'],
      dtype='object')

The line below, commented out so it doesn't automatically trigger each time, creates the datasets needed for the lookup tool.

In [51]:
# the csv is for the detailed pages for each NH
#  st_cms.to_csv('July_2019_project/Provider_info_w_cost_report_data.csv', index=False)

#the JSON is only for the lookup table on the homepage. Warning: Pandas json conversion may be wonky so you may have to wing it with an online converter
# st_cms[['Provider Name', 'Provider City', 'Number of Certified Beds', 'Overall Rating','Federal Provider Number']].to_json("July_2019_project/Provider_info_w_cost_report_data.json", orient='values')